In [1]:
# !pip install jieba

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
import gc
import time
import os

In [4]:
from random import randint

import numpy as np
import torch
import pandas as pd
import re
import os
os.environ["CUDA_VISIBLE_DEVICES"] ='0'

In [5]:
pub_info = pd.read_pickle('./pkl/pub_info.pkl')

In [6]:
pub_info.head()

,abstract,authors,keywords,paper_id,title,venue,year,orgs
0,Extremely long states of bound solitons consis...,"[Ya Liu, Xin Zhao, Jiansheng Liu, Guoqing Hu]","[carbon nanotubes, fibre lasers, laser cavity ...",0009qJgC,Generating ultra-long bound soliton sequences ...,Lasers and Electro-Optics,2014,"[Sch. of Electron. & Inf. Eng., Beihang Univ.,..."
1,,"[Hao He, Jun Yang]","[different part, data graph, variety ofstructu...",000BMUBY,Multiresolution Indexing of XML for Frequent Q...,icde,2004,[]
2,The Yellow River (YR) supplies a large amount ...,"[Weifeng Yang, Min Chen, Guangxue Li, Jianping...","[13C, 15N, East China Sea, Trace element, Yell...",000Fmw78,Relocation of the Yellow River as revealed by ...,Marine Pollution Bulletin,2009,[State Key Laboratory of Marine Environmental ...
3,Our findings suggest that Th17 cells may be in...,"[Xiaowei Wang, Xiaojun Chen, Heng Tang, Jifeng...","[Mycoplasma Pneumoniae, Th17 cells, children, ...",000HsdYy,Increased Frequency of Th17 Cells in Children ...,Journal of clinical laboratory analysis,2016,"[Department of Blood Transfusion, Nanjing Chil..."
4,,"[jun chen, xianran xing, andrew d watson, wei ...",[],000NKgQT,Rapid Synthesis of Multiferroic BiFeO 3 Single...,Chemistry of Materials,2007,"[, , , , , , , , ]"


### prepare

In [7]:
pub_info.isnull().sum()

abstract     0
authors      0
keywords    70
paper_id     0
title        0
venue        0
year         0
orgs         0
dtype: int64

In [8]:
pub_info['keywords'] = pub_info['keywords'].apply(lambda x: [] if type(x) == float else x)

In [9]:
# max_len=256
# def func(s):
#     s = re.sub(r'[^\w\s]', ' ', s)
#     wds = s.split()[:max_len]
#     return ' '.join(wds)

In [10]:
# cols = ['abstract', 'title', 'venue']
# for c in cols:
#     print(c)
#     pub_info[c] = pub_info[c].apply(func)

In [11]:
def func2(l):
    return ', '.join(l)

In [12]:
cols = ['authors', 'keywords', 'orgs']
for c in cols:
    print(c)
    pub_info[c] = pub_info[c].apply(func2)

authors
keywords
orgs


In [13]:
pub_info.head()

,abstract,authors,keywords,paper_id,title,venue,year,orgs
0,Extremely long states of bound solitons consis...,"Ya Liu, Xin Zhao, Jiansheng Liu, Guoqing Hu","carbon nanotubes, fibre lasers, laser cavity r...",0009qJgC,Generating ultra-long bound soliton sequences ...,Lasers and Electro-Optics,2014,"Sch. of Electron. & Inf. Eng., Beihang Univ., ..."
1,,"Hao He, Jun Yang","different part, data graph, variety ofstructur...",000BMUBY,Multiresolution Indexing of XML for Frequent Q...,icde,2004,
2,The Yellow River (YR) supplies a large amount ...,"Weifeng Yang, Min Chen, Guangxue Li, Jianping ...","13C, 15N, East China Sea, Trace element, Yello...",000Fmw78,Relocation of the Yellow River as revealed by ...,Marine Pollution Bulletin,2009,State Key Laboratory of Marine Environmental S...
3,Our findings suggest that Th17 cells may be in...,"Xiaowei Wang, Xiaojun Chen, Heng Tang, Jifeng ...","Mycoplasma Pneumoniae, Th17 cells, children, r...",000HsdYy,Increased Frequency of Th17 Cells in Children ...,Journal of clinical laboratory analysis,2016,"Department of Blood Transfusion, Nanjing Child..."
4,,"jun chen, xianran xing, andrew d watson, wei w...",,000NKgQT,Rapid Synthesis of Multiferroic BiFeO 3 Single...,Chemistry of Materials,2007,", , , , , , , ,"


### embedding

In [14]:
# !unzip ../../../multi/bert-large-nli-mean-tokens.zip -d ./bert-large-nli-mean-tokens/

In [15]:
import torch
torch.cuda.is_available()

True

In [16]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('./bert-large-nli-mean-tokens')

In [17]:
# model = model.cuda()

In [18]:
pub_info_embed = pd.DataFrame()

In [19]:
cols = ['abstract', 'authors', 'keywords', 'title', 'venue', 'orgs']
for c in cols:
    print(c)
    pub_info_c = list(pub_info[c].values)
    pub_info_c_embed = model.encode(pub_info_c, show_progress_bar=True, batch_size=128)
    pub_info_embed[c] = pub_info_c_embed
    display(pub_info_embed.head(2))

Batches:   0%|          | 0/2119 [00:00<?, ?it/s]

abstract


Batches: 100%|██████████| 2119/2119 [1:01:41<00:00, 23.88s/it]


,abstract
0,"[0.79583216, -0.29403698, 0.26839104, 0.358583..."
1,"[-0.8647487, 0.53667474, 0.43480504, 0.0010895..."


Batches:   0%|          | 0/2119 [00:00<?, ?it/s]

...

,abstract,authors,keywords,title,venue
0,"[0.79583216, -0.29403698, 0.26839104, 0.358583...","[-0.48936775, -0.57858217, 0.71193564, -0.1749...","[0.43632925, -0.9237635, 0.40525427, 0.2522947...","[0.3280066, -0.050479, 0.42760244, 0.051524464...","[-0.26035744, -0.79420173, -0.41245815, -0.312..."
1,"[-0.8647487, 0.53667474, 0.43480504, 0.0010895...","[-0.3726856, -0.067486644, 0.4936543, -0.15705...","[-0.03109114, -0.5499945, 0.5445511, -0.013458...","[-0.1385318, -0.29783824, 0.8824967, -0.348692...","[-0.82357174, -0.18565406, 0.2076622, -0.47456..."


Batches:   0%|          | 0/2119 [00:00<?, ?it/s]

orgs


Batches: 100%|██████████| 2119/2119 [51:07<00:00, 82.66s/it] 


,abstract,authors,keywords,title,venue,orgs
0,"[0.79583216, -0.29403698, 0.26839104, 0.358583...","[-0.48936775, -0.57858217, 0.71193564, -0.1749...","[0.43632925, -0.9237635, 0.40525427, 0.2522947...","[0.3280066, -0.050479, 0.42760244, 0.051524464...","[-0.26035744, -0.79420173, -0.41245815, -0.312...","[-0.20673345, -0.64450264, 0.40301833, -0.4305..."
1,"[-0.8647487, 0.53667474, 0.43480504, 0.0010895...","[-0.3726856, -0.067486644, 0.4936543, -0.15705...","[-0.03109114, -0.5499945, 0.5445511, -0.013458...","[-0.1385318, -0.29783824, 0.8824967, -0.348692...","[-0.82357174, -0.18565406, 0.2076622, -0.47456...","[-0.8647487, 0.53667474, 0.43480504, 0.0010895..."


In [21]:
pub_info_embed.to_pickle('./pkl/pub_info_embed_bert.pkl')